In [1]:
import torch
import torchvision
import numpy as np

In [2]:
from cifar10models import *

In [3]:
model = ResNet18()

In [4]:
model.load_state_dict(torch.load('../../../../share/cuvl/weights/cifar10/resnet18_epoch_347_acc_94.77.pth'))

In [5]:
from attacks import ifgsm

In [6]:
import torchvision.datasets as dset

In [7]:
def _data_transforms_cifar10():
  CIFAR_MEAN = [0.49139968, 0.48215827, 0.44653124]
  CIFAR_STD = [0.24703233, 0.24348505, 0.26158768]

  train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
  ])

  valid_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
    ])
  return train_transform, valid_transform

In [8]:
train_transform, valid_transform = _data_transforms_cifar10()

In [9]:
train_data = dset.CIFAR10(root='../data', train=True, download=True, transform=train_transform)

Files already downloaded and verified


In [10]:
num_train = len(train_data)
indices = list(range(num_train))
split = int(np.floor(0.5 * num_train))

In [11]:
train_tensors = []
train_label = []

In [12]:
for i in range(num_train):
    train_tensors.append(train_data[i][0])
    train_label.append(train_data[i][1])

In [13]:
train_tensors[5].size()

torch.Size([3, 32, 32])

In [14]:
train_data.__getitem__(0)

(tensor([[[-1.2114, -0.6716, -0.2430,  ..., -0.2589, -1.9892, -1.9892],
          [-0.6081, -0.3065, -0.1636,  ..., -0.4970, -1.9892, -1.9892],
          [-0.3700,  0.0269, -0.0207,  ..., -0.9256, -1.9892, -1.9892],
          ...,
          [ 0.8524,  0.9952,  1.2175,  ..., -0.0366, -1.9892, -1.9892],
          [-1.9892, -1.9892, -1.9892,  ..., -1.9892, -1.9892, -1.9892],
          [-1.9892, -1.9892, -1.9892,  ..., -1.9892, -1.9892, -1.9892]],
 
         [[-1.5454, -1.1749, -0.8206,  ..., -0.8045, -1.9802, -1.9802],
          [-1.1105, -0.9656, -0.8528,  ..., -0.9817, -1.9802, -1.9802],
          [-0.9333, -0.7079, -0.7401,  ..., -1.3038, -1.9802, -1.9802],
          ...,
          [ 0.3068,  0.4196,  0.7256,  ..., -0.4985, -1.9802, -1.9802],
          [-1.9802, -1.9802, -1.9802,  ..., -1.9802, -1.9802, -1.9802],
          [-1.9802, -1.9802, -1.9802,  ..., -1.9802, -1.9802, -1.9802]],
 
         [[-1.5871, -1.3622, -1.0924,  ..., -1.0774, -1.7070, -1.7070],
          [-1.3322, -1.2872,

In [15]:
import pickle

In [61]:
num_train = len(train_data)
indices = list(range(num_train))
split = int(np.floor(0.5 * num_train))

train_queue = torch.utils.data.DataLoader(
  train_data, batch_size=64,
  sampler=torch.utils.data.sampler.SubsetRandomSampler(indices[:]),
  pin_memory=True, num_workers=2)

In [17]:
import torch.nn as nn

def ifgsm(model, X, y, niters=10, epsilon=0.01, visualize=False):
    X_pert = X.clone()
    X_pert.requires_grad = True
    
    for _ in range(niters):
        output_perturbed = model(X_pert)
        loss = nn.CrossEntropyLoss()(output_perturbed, y)
        loss.backward()
        pert = epsilon * X_pert.grad.detach().sign()

        # add perturbation
        X_pert = X_pert.detach() + pert
        X_pert.requires_grad = True
        
        # make sure we don't modify the original image beyond epsilon
        X_pert = (X_pert.detach() - X.clone()).clamp(-epsilon, epsilon) + X.clone()
        X_pert.requires_grad = True
        
        # adjust to be within [-1, 1]
        X_pert = X_pert.detach().clamp(-1, 1)
        X_pert.requires_grad = True
        
    return X_pert

In [62]:
from torch.autograd import Variable
perturbed_images = []
perturbed_labels = []
for step, (input, target) in enumerate(train_queue):
    input = Variable(input)
    target = Variable(target)
    
    input = ifgsm(model, input, target, niters=3)
    perturbed_images.append(input)
    perturbed_labels.append(target)

Process Process-12:
Process Process-11:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/share/apps/anaconda3/5.2.0/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/share/apps/anaconda3/5.2.0/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/share/apps/anaconda3/5.2.0/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/share/apps/anaconda3/5.2.0/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/share/apps/anaconda3/5.2.0/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/share/apps/anaconda3/5.2.0/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  

KeyboardInterrupt: 

In [67]:
input = train_data[-1]
input = ifgsm(model, train_data[-1][0].unsqueeze(0), torch.tensor([train_data[-1][1]]), niters=3)

In [72]:
pert_lbl.shape

(49999,)

In [73]:
pert_imgs = torch.cat([torch.tensor(pert_imgs), input])
pert_lbl = torch.cat([torch.tensor(pert_lbl), torch.tensor([train_data[-1][1]])])

In [34]:
concat = torch.cat(perturbed_images)

In [39]:
pert_imgs = concat.detach().numpy()

In [46]:
pert_lbl = torch.cat(perturbed_labels).detach().numpy()

In [74]:
np.save('resnet_attacked.npy', pert_imgs.detach().numpy())

In [75]:
np.save('resnet_lbls.npy', pert_lbl.detach().numpy())

In [78]:
len(pert_imgs.detach().numpy())

50000